In [1]:
import logging
import numpy as np
import pandas as pd
from tqdm import tqdm
from pax import core
import multihist
import sys
import os
import scipy.integrate as integrate
from runDB import get_name
import stat
import shutil

In [2]:
def get_run(run):
    mypax = core.Processor(config_names='XENON1T', config_dict={
        'pax': {
            'plugin_group_names': ['input', 'preprocessing'],
            'preprocessing':      ['CheckPulses.SortPulses',
                                   'CheckPulses.ConcatenateAdjacentPulses',],
            'input':              'Zip.ReadZipped',
            'encoder_plugin':     None,
            #'decoder_plugin':     'BSON.DecodeZBSON',
            'input_name':          run
        }
    })
    return mypax

In [3]:
from spe_acceptance import get_name
from spe_acceptance import rawdata_dir
LED_run=18287
run_name = get_name(LED_run)
run_name=os.path.join(rawdata_dir, run_name)
processor = get_run(run_name)

processor MainProcess L66 INFO This is PAX version 6.8.0, running with configuration for XENON1T.
ReadZipped MainProcess L102 INFO InputFromFolder: Selecting file /project/lgrandi/xenon1t/spe_acceptance/rawdata/180409_1626/XENON1T-18287-000000000-000000999-000001000.zip (number 1/197 in folder) for reading


In [ ]:
# generator used to loop over events
def get_events(RUN):
    for event in RUN.get_events():
        event = RUN.process_event(event)
        yield event

In [ ]:
event_generator = get_events(processor)
N = processor.get_events()

n_channels=254
channels=np.ones(n_channels)
medians = []
baselines = []
diff_1=[]
diff_2=[]

for i in tqdm(N):
    event = next(event_generator)
    
    for ch, p in enumerate(event.pulses):
        w=p.raw_data
        channels[ch]=p.channel
        baseline1 = np.median(p.raw_data)
        baseline2 = p.baseline
        medians.append(baseline1)
        baselines.append(baseline2)
        diff_1.append(baseline1-w)
        diff_2.append(baseline2-w)
        
    

997it [00:38, 37.52it/s]ReadZipped MainProcess L102 INFO InputFromFolder: Selecting file /project/lgrandi/xenon1t/spe_acceptance/rawdata/180409_1626/XENON1T-18287-000001000-000001999-000001000.zip (number 2/197 in folder) for reading
2000it [01:14, 37.95it/s]ReadZipped MainProcess L102 INFO InputFromFolder: Selecting file /project/lgrandi/xenon1t/spe_acceptance/rawdata/180409_1626/XENON1T-18287-000002000-000002999-000001000.zip (number 3/197 in folder) for reading
2996it [01:53, 33.54it/s]ReadZipped MainProcess L102 INFO InputFromFolder: Selecting file /project/lgrandi/xenon1t/spe_acceptance/rawdata/180409_1626/XENON1T-18287-000003000-000003999-000001000.zip (number 4/197 in folder) for reading
3997it [02:29, 37.25it/s]ReadZipped MainProcess L102 INFO InputFromFolder: Selecting file /project/lgrandi/xenon1t/spe_acceptance/rawdata/180409_1626/XENON1T-18287-000004000-000004999-000001000.zip (number 5/197 in folder) for reading
4999it [03:06, 37.57it/s]ReadZipped MainProcess L102 INFO Inp

In [ ]:
plt.hist(diff_1, bins=1000)
plt.title("With Median")
plt.show()

In [ ]:
plt.hist(diff_2, bins=1000)
plt.title("With Pax")
plt.show()